# Umair Ahmad
#21i-2081
# Umair.Ahmad@nu.edu.pk

#Mount the Google Drive to Load Datasets and save tranined checkpoitns 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Required Libraries

In [2]:
!pip install transformers==4.4.0
!pip install datasets==1.4.1
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install urduhack

     |████████████████████████████████| 2.1MB 31.0MB/s 
     |████████████████████████████████| 3.3MB 44.7MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 
     |████████████████████████████████| 194kB 34.2MB/s 
     |████████████████████████████████| 122kB 42.1MB/s 
     |████████████████████████████████| 245kB 48.8MB/s 
     |████████████████████████████████| 1.9MB 30.4MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149822 sha256=767338b99d64319859308eb916619f1e094dcfd120075ccbbf60727f4a1733a9
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 112kB 36.1MB/s 
     |████████████████████████████████| 3.4MB 36.3MB/s 
     |████████████████████████████████| 686kB 38.8MB/s 
  Found existing installation: tensorflow-datasets 4.0

In [3]:
from datasets import Dataset
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import normalize_whitespace
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import torchaudio
import librosa
import numpy as np
import random
from collections import defaultdict
import pandas as pd

# Loading and Prerocessing the Urdu transcriptions, Convert the dataset into Apache Arrow 

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/AudioData/audio transcriptions (1).csv")
lst=[]
for x in data["text"]:
  output = remove_punctuation(str(x))
  output=remove_accents(output)
  output=replace_numbers(output)
  output = normalize_whitespace(output)
  lst.append(output)
data["text"]=lst
dataframe=data.head()
data = Dataset.from_pandas(data)
print(data)
dataframe

Dataset({
    features: ['Path', 'text'],
    num_rows: 2999
})


,Path,text
0,/content/drive/MyDrive/AudioData/Audio Files/1...,اب دوبارہ آ رہی ہےموڈیم آن ہو گیا ہے
1,/content/drive/MyDrive/AudioData/Audio Files/0...,آپ صرف اتنی بات کریں گے تو آپ کو
2,/content/drive/MyDrive/AudioData/Audio Files/3...,جی واپس آؤ
3,/content/drive/MyDrive/AudioData/Audio Files/2...,اپنا خیال رکھئے گا الله حافظ
4,/content/drive/MyDrive/AudioData/Audio Files/d...,جی آویلیبل ہے محترم ایم بی ایم بی


In [56]:
#Test_Data
colnames=['text'] 
test_data=pd.read_csv("/content/drive/MyDrive/AudioData/Test_Data_Transcript.csv",names=colnames,header=None)
paths=[]
for x in range(1,len(test_data["text"])+1):
  temp=str("/content/drive/MyDrive/AudioData/Test Dataset/")+str(x)+".wav"
  paths.append(temp)
test_data['Path'] = paths
lst=[]
for x in test_data["text"]:
  output = remove_punctuation(str(x))
  output=remove_accents(output)
  output=replace_numbers(output)
  output = normalize_whitespace(output)
  lst.append(output)
test_data["text"]=lst
dataframe=test_data.head()
test_data = Dataset.from_pandas(test_data)
print(test_data)
dataframe

Dataset({
    features: ['text', 'Path'],
    num_rows: 169
})


,text,Path
0,جی سر میں ڈیٹیلز چیک کرتا ہوں کائنڈلی ویٹ کیجی...,/content/drive/MyDrive/AudioData/Test Dataset/...
1,اسلام علیکم کر رہا ہوں جی فرمائے میں آپکی کیا ...,/content/drive/MyDrive/AudioData/Test Dataset/...
2,میں سر دیکھ لیتا ہوں جس نمبر سے کال کر رہے ہیں...,/content/drive/MyDrive/AudioData/Test Dataset/...
3,جی جس نمبر سے آپ کال کر رہے ہیں اسی نمبر پہ اش...,/content/drive/MyDrive/AudioData/Test Dataset/...
4,مزید ویٹ کیجیے گا سر,/content/drive/MyDrive/AudioData/Test Dataset/...


# Genreted the urdu Vocab also added the required Tokens i.e  "[UNK]" "[PAD]" and replace the space with "|" for visibility 

In [14]:
dict1 = {0: ' ',1: 'ا',2: 'آ',3: 'ب',4: 'پ',5: 'ت',6: 'ٹ',7: 'ث',8: 'ج',9: 'چ',10: 'ح',11: 'خ',12: 'د',13: 'ڈ',14: 'ذ',15: 'ر',16: 'ڑ',17: 'ز',18: 'ژ',19: 'س',20: 'ش',21: 'ص',22: 'ض',23: 'ط',24: 'ظ',25: 'ع',26: 'غ',27: 'ف',28: 'ق',29: 'ک',30: 'گ',31: 'ل',32: 'م',33: 'ن',34: 'ں',35: 'و',36: 'ہ',37: 'ھ',38: 'ء',39: 'ئ',40: 'ی',41: 'ے'}
vocab_dict=defaultdict(int)
for key,value in dict1.items():
  vocab_dict[value]=key
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

44

# Declare the Tokenizer , Feature Extracter, and Processor
## Also saved this in drive for later use

In [15]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file,ensure_ascii = False)
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-Urdu")

# Reading audio files and mapping to 16000 sampling rate

In [18]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 16_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
data = data.map(speech_file_to_array_fn, remove_columns=data.column_names)

In [57]:
test_data=test_data.map(speech_file_to_array_fn, remove_columns=test_data.column_names)

In [ ]:
data = data.map(resample, num_proc=4)

In [58]:
test_data = test_data.map(resample, num_proc=4)

# Genrated a Sample for Visualization

In [ ]:
rand_int = random.randint(0, len(data)-1)
print("Target text:", data[rand_int]["target_text"])
print("Input array shape:", np.asarray(data[rand_int]["speech"]).shape)
print("Sampling rate:", data[rand_int]["sampling_rate"])
sample=data[rand_int]["speech"]
sample_label=data[rand_int]["target_text"]
print(sample)
print(sample_label)

Target text: اپنا خیال رکھنا اللہ حافظ
Input array shape: (34400,)
Sampling rate: 16000
[0.0084228515625, 0.00244140625, -0.00738525390625, -0.014312744140625, -0.01629638671875, -0.0166015625, -0.017181396484375, -0.016357421875, -0.013031005859375, -0.010467529296875, -0.01165771484375, -0.013519287109375, -0.011474609375, -0.008636474609375, -0.01470947265625, -0.033111572265625, -0.052734375, -0.05926513671875, -0.05169677734375, -0.042083740234375, -0.03936767578125, -0.03973388671875, -0.03582763671875, -0.0299072265625, -0.031280517578125, -0.0423583984375, -0.053436279296875, -0.053436279296875, -0.04205322265625, -0.02886962890625, -0.021942138671875, -0.0203857421875, -0.017791748046875, -0.010711669921875, -0.002532958984375, 0.001068115234375, -0.000732421875, -0.00274658203125, 0.000640869140625, 0.008941650390625, 0.0174560546875, 0.023284912109375, 0.0279541015625, 0.03265380859375, 0.03497314453125, 0.0330810546875, 0.03082275390625, 0.033843994140625, 0.04156494140625,

 # Checking that all files have the correct sampling rate

In [26]:
def prepare_dataset(batch):
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
data = data.map(prepare_dataset, remove_columns=data.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [27]:
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# Importing Data_collator with CTC Padding

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# Loading WER Matrix 

In [42]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [31]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Creating Instance of PreTranined "facebook/wav2vec2-large-xlsr-53" Model and tuning Parameters 

In [32]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

# Loading XLSR Transformers for Fine Tuning

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/wav2vec2-large-xlsr-Urdu",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=300,
  save_total_limit=3,
)

# Split the dataset for Training and Validation during training

In [ ]:
data=data.train_test_split(test_size=0.1)
data["train"]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 2699
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=processor.feature_extractor,
)

# Training Started 

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
200,12.024900,3.689662,1.000000,17.267700,17.374000
400,3.690500,3.621338,1.000000,17.487600,17.155000
600,3.467300,3.893539,1.000000,17.569200,17.075000
800,3.288800,3.646878,1.000000,17.428800,17.213000
1000,3.185800,3.674290,1.000000,17.086700,17.558000
1200,3.110000,3.447118,0.999448,17.080300,17.564000
1400,2.809500,2.867014,0.965746,17.125800,17.517000
1600,2.381200,2.861734,0.918232,17.121400,17.522000
1800,2.013300,2.491365,0.856354,17.054800,17.590000
2000,1.728000,2.847434,0.865193,17.245000,17.396000


TrainOutput(global_step=2520, training_loss=3.280703850397988, metrics={'train_runtime': 6374.9829, 'train_samples_per_second': 0.395, 'total_flos': 5.271714946009628e+18, 'epoch': 29.99, 'init_mem_cpu_alloc_delta': 313314, 'init_mem_gpu_alloc_delta': 1261935616, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1939091, 'train_mem_gpu_alloc_delta': 3785499136, 'train_mem_cpu_peaked_delta': 35843140, 'train_mem_gpu_peaked_delta': 958381568})

# As we can clearly see the checkpoint 2200 provide the best Training Loss	Validation Loss	and Wer
## Checkpoint 2400 started to overfit so 2200 check point is best and saved in the Drive
## Now we need to load the 2200 checkpoint from the drive for Testing

In [34]:
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-Urdu/checkpoint-2200").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-Urdu")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
input_dict = processor(sample, return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


# Prediction of the One Sample

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(sample_label)


Prediction:
ک اپنا خال ہتھے ہ الہ افظ

Reference:
اپنا خیال رکھنا اللہ حافظ


# Testing Phase 

In [60]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

result = test_data.map(evaluate, batched=True, batch_size=8)

# 83.467095 is the WER Score of Test Dataset

In [62]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["target_text"])))

WER: 83.467095


# Visualization of the Predicted and Real Labels

In [65]:
dataframe=Dataset.to_pandas(result)
dataframe = dataframe[["pred_strings", "target_text"]]
dataframe

,pred_strings,target_text
0,جی سر میٹد سے کتا ہو کائنڈی ویٹٹجیجئے,جی سر میں ڈیٹیلز چیک کرتا ہوں کائنڈلی ویٹ کیجی...
1,السلام علیکمک فربائم س کیام مد ک ستا,اسلام علیکم کر رہا ہوں جی فرمائے میں آپکی کیا ...
2,میں سر دنتا ہوںجس نمبر سے کال کر رہے ہیں سنوپ ...,میں سر دیکھ لیتا ہوں جس نمبر سے کال کر رہے ہیں...
3,جیڈس نمبر سےے آپ کال کر رہ ہی اسنورپ اشو آ رہا ہے,جی جس نمبر سے آپ کال کر رہے ہیں اسی نمبر پہ اش...
4,موجی ویٹ کیجئے گا سر,مزید ویٹ کیجیے گا سر
...,...,...
164,جیٹٹی,جی ٹو فور ٹو تھری سکس
165,اوچاجسر,فور سیون فائیو زیرو ٹو فور ٹو تھری سکس جی سر
166,ڈوزنگ بت,براؤزنگ بہت کر رہا ہے
167,ٹھیک ہے کشر میںوز کر رینے وائس کو سار,ٹھیک ہے کس شہر میں یوز کر رہے ہیں ڈیوائسس کو س...
